# Image Steganography

In [9]:
# Import all the required libarries 

import cv2 
import numpy as np 
import types 
import matplotlib.pyplot as plt 
%matplotlib inline

In [10]:
def messageToBinary(message):
    if type(message) == str: 
        return ''.join([format(ord(i),'08b') for i in message])
    elif type(message) == bytes or type(message) == np.ndarray:
        return [format(i,'08b') for i in message]
    elif type(message) == int or type(message) == np.uint8:
        return format(message,'08b')
    else:
        raise TypeError('Input Type not supported')

In [11]:
def hideData(image,secret_message):

    # calculate the maximum bytes to encode 
    n_bytes = image.shape[0] * image.shape[1] * 3 // 8 
    print('Maximum bytes to encode: ', n_bytes)

    # Check if the number of bytes to encode is less than the maximum bytes in 
    if len(secret_message) > n_bytes:
        raise ValueError('Error encountered insufficient bytes')
    
    secret_message += '$$$$$'

    data_index = 0 
    # convert input data to binary format using messageToBinary() function
    binary_secret_msg = messageToBinary(secret_message)

    data_len = len(binary_secret_msg)
    for values in image:
        for pixel in values:
            # convert RGB values to binary format 
            r , g ,b =messageToBinary(pixel)

            # Modify the least significant bit only if there is still data
            if data_index < data_len:
                # hide the data into least significant bit of red pixel 
                pixel[0] = int(r[:-1] + binary_secret_msg[data_index],2)
                data_index += 1 
            
            if data_index < data_len:
                # hide the data into least significant bit of green pixel
                pixel[1] = int(g[:-1] + binary_secret_msg[data_index],2)
                data_index += 1
            
            if data_index < data_len:
                # hide the data into least significant bit of blue pixel
                pixel[2] = int(b[:-1] + binary_secret_msg[data_index],2)
                data_index += 1
            
            # if data is encoded, just break out of the loop
            if data_index >= data_len:
                break 
    return image

In [12]:
def showData(image):

    binary_data = ''
    for values in image:
        for pixel in values:
            r , g , b = messageToBinary(pixel)
            binary_data += r[-1]
            binary_data += g[-1]
            binary_data += b[-1]
    
    # split by 8-bits 
    all_bytes = [binary_data[i:i+8] for i in range(0,len(binary_data),8)]

    # convert from bits to characters 
    decoded_data = ''
    for byte in all_bytes:
        decoded_data += chr(int(byte,2))
        if decoded_data[-5:] == '$$$$$':
            break 
    print(decoded_data)
    return decoded_data[:-5]


In [13]:
# Encode data into image
def encode_text():
    image_name = input('Enter image name (with extension)')
    image = cv2.imread(image_name)

    print('the shape of the image is ',image.shape)
    print('The original image is as shown below')

    resized_image = cv2.resize(image,(500,500))
    plt.imshow(cv2.cvtColor(resized_image,cv2.COLOR_BGR2RGB))

    data = input('Enter data to be encoded')
    if (len(data) == 0 ):
        raise ValueError('Data is Empty')
    
    filename = input('Enter the name of the new encoded image (with extension)')
    encoded_image = hideData(image,data)
    cv2.imwrite(filename,encoded_image)

In [14]:
# Decode the data in the image

def decode_text():
    # read the image that contains the hidden image 
    image_name = input('Enter the name of the steganographed image that you like')
    image = cv2.imread(image_name)

    print('The steganographed image is as shown below:')
    resized_image = cv2.resize(image,(500,500))
    plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2GRAY))

    text = showData(image)
    return text

In [15]:
def Steganography():

    a = input('Image Steganography \n 1. Encode the data \n 2 Decode the data')
    userinput = int(a)
    if(userinput == 1):
        print('\n Encoding..')
        encode_text()
    elif(userinput == 2):
        print('\n Decoding..')
        print('Decoded message is ' + decode_text())
    else:
        raise Exception('Enter correct input ')
    
Steganography()


 Decoding..
The steganographed image is as shown below:


In [ ]:
Steganography()